In [1]:
import ROOT as R
#R.ROOT.EnableImplicitMT()

import os
from pprint import pprint
import pandas as pd 


Welcome to JupyROOT 6.19/01


In [5]:
import PyRDF
PyRDF.use("spark")
PyRDF.include_headers("headers.hh")

print(PyRDF.current_backend.supported_operations)

['Define', 'Filter', 'Histo1D', 'Histo2D', 'Histo3D', 'Profile1D', 'Profile2D', 'Profile3D', 'Count', 'Sum', 'Fill', 'Snapshot', 'Graph', 'AsNumpy']


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cvmfs/sft-nightlies.cern.ch/lcg/nightlies/dev3python3/Sun/spark/2.4.4-cern2/x86_64-centos7-gcc8-opt/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cvmfs/sft.cern.ch/lcg/releases/hadoop/2.7.5.1-79196/x86_64-centos7-gcc8-opt/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
20/01/19 15:44:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
20/01/19 15:44:39 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cl

In [2]:
#R.gInterpreter.ProcessLine(".L headers.hh")

0

In [2]:
import latinos_rdf as lrdf

In [3]:
#samples = ["DATA","VBS", "VBF-V", "Wjets", "singleTop", "ttbar",  "VV","DY","FakeQCD", "VVV"]
samples = ["VBS", "Wjets", "DY", "singleTop", "ttbar"]
cut = "muon_looseVBS"
output_basedir = "/eos/user/d/dmapelli/public/latino/Full2017_v1_200118/"

In [6]:
columns_MC = [ "events"]
#columns_MC = ["weight", "XSWeight", "TriggerEffWeight_1l", "btagSF"] + columns
#columns_DATA = columns
columns_MC = [ cut+"_"+c for c in columns_MC]

In [7]:
joblist = []

for sample in samples:
    #trees = lrdf.build_dataframe("lowenergy", sample, R, "root")
    print(sample)
    trees, nfiles = lrdf.build_dataframe("Full2017v6",  sample, PyRDF , "pyrdf")
    for tree, nfile in zip(trees,nfiles):
        joblist.append((tree,nfile))
        
######## Now sort by number of files
jobslist = sorted(joblist, key=lambda v: v[1], reverse=True)
    


VBS
1 [1]
-,  1
Wjets
1 [1]
-,  1
DY
3 [8, 24, 58]
-,  8
-,  24
-,  58
singleTop
1 [1]
-,  1
ttbar
1 [1]
-,  1


In [8]:
jobslist

[(name: supercut
  parent: None
  cut: METFilter_MC && nLepton==1 && Lepton_pt[0]>30 && VBS_category==1
  vars: 
  aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
  weight: (puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF*(Sum(GenPart_pdgId == 22 && TMath::Odd(GenPart_statusFlags) && GenPart_pt > 20.) == 0))*((1.)*(((0.632927+0.0456956*gen_ptll-0.00154485*gen_ptll*gen_ptll+2.64397e-05*gen_ptll*gen_ptll*gen_ptll-2.19374e-07*gen_ptll*gen_ptll*gen_ptll*gen_ptll+6.99751e-10*gen_ptll*gen_ptll*gen_ptll*gen_ptll*gen_ptll)*(gen_ptll>0)*(gen_ptll<100)+(1.41713-0.00165342*gen_ptll)*(gen_ptll>=100)*(gen_ptll<300)+1*(gen_ptll>=300))))
  -----------------------------------------------------------------------------

In [ ]:
########
dfs = []
for tree,_ in jobslist:
        print(tree)
        # Choose the right output columns
        if sample == "DATA": outputcols = columns_DATA
        else: outputcols = columns_MC

        df = pd.DataFrame(tree[cut].rdf_node.AsNumpy(columns=outputcols))
        dfs.append(df)

    all_df = dfs[0]
    for df in dfs[1:]:
        all_df = all_df.append(df)  

    all_df = all_df[ ( (all_df.lep_flavour == 11) & (all_df.lep_pt > 40))  | (all_df.lep_flavour == 13)]

    all_df.to_pickle(output_basedir + sample + ".pkl")

In [7]:
df

,muon_looseVBS_var_events
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1
